- version

ver 1: singlefold, train using only 30 features >>> lb 0.103

ver 5: 5 GroupKFolds (actually train just one fold), make oof, train using all features

ver 6: just infer with ver5 model >>>  lb0.132

ver 7: wanna solve memory error... >>> failed

ver 8-11: train fold1-4

ver 12: infer with 5 models from ver5, 8-11 >>> made mistake, infer by just one model

ver 14: infer with 5 models from ver5, 8-11 >>> lb0.136

ver 16: median ensemble of ver 14 >>> lb0.135

ver 17: train with categorical feature 'investment_id'

- reference

https://www.kaggle.com/nischaydnk/different-splits-for-the-competition-parquets

https://www.kaggle.com/columbia2131/speed-up-reading-csv-to-pickle

https://www.kaggle.com/currypurin/ubiquant-simple-lgbm-custom-metric

In [ ]:
# import libs
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
import glob
import pickle

In [ ]:
##### config #####
# select inference only or not, debug or not
infer_only = False
debug = False

# cols
basic_cols = ['row_id', 'time_id', 'investment_id', 'target']
num_feat = 300
features = [f'f_{i}' for i in range(num_feat)]
features.append('investment_id')

# params for lgbm
params = {
        'random_state':2022, 
        'verbosity': -1,
        'metrics': 'rmse',
    }
num_round = 500

# fold
num_folds = 5
folds = [0] #[0,1,2,3,4]

# debug
if debug:
    num_round = 1

In [ ]:
models = []

# if train and infer
if not infer_only:
    oof = np.zeros((3141410))
    for fold in folds:
        print('='*40)
        print(f'fold{fold}')
        # to avoid memory error, load df and delete repeatedly 
        train_df = pd.read_parquet(f'../input/different-splits-for-the-competition-parquets/df_gs_{num_folds}folds.parquet')
        tr_y = train_df[train_df['fold']!=fold]['target'].values
        tr_x = train_df[train_df['fold']!=fold][features].values
        val_idx = np.array(train_df['fold']==fold).tolist()
        val_y = train_df.iloc[val_idx]['target'].values
        val_x = train_df.iloc[val_idx][features].values
        
        del train_df
        gc.collect()      

        lgb_train = lgb.Dataset(tr_x, tr_y, categorical_feature=[300],)
        lgb_eval = lgb.Dataset(val_x, val_y, categorical_feature=[300],)
        model = lgb.train(params, lgb_train, valid_sets=lgb_eval,
                          verbose_eval=50,
                          num_boost_round = num_round,
                          early_stopping_rounds=100)
        oof_preds = model.predict(val_x)
        oof[val_idx] = oof_preds
        models.append(model)
        pickle.dump(model, open(f'lgb_fold{fold}.pkl', 'wb'))
        
        del lgb_train, lgb_eval, tr_x, tr_y, val_x, val_y, model, val_idx
        gc.collect()
    #targets = pd.read_csv('../input/ubiquant-market-prediction/train.csv', usecols=['target'])
    np.save('oof', oof)
# if just infer
else:
    model_paths = glob.glob(f'../input/ubiquant-lgb-models-1901/lgb_fold*.pkl')
    for model_path in model_paths:
        models.append(pickle.load(open(model_path, 'rb')))
        print(f'loaded {model_path}')

In [ ]:
# inference
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    preds = []
    for model in models:
        preds.append(model.predict(test_df[features].values))
    sample_prediction_df['target'] = np.median(preds, axis=0)  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions